In [1]:
###########################BACKGROUND ALGORITHMS NEEDED ################################################################
#gcd is already implimented into sage 
#Prime sieve is already in sage as primes(B)

def modinv(a,n):#computes a^-1 mod n using extended euclidean algorithm
    a%=n
    r=(0,1)
    #s=(1,0)
    N=(n,a)
    #at all times, we should have N[0]=r[0]*a+s[0]*n
    while N[1]!=0:
        d=N[0]//N[1] #
        r=(r[1],r[0]-d*r[1])
        #s=(s[1],s[0]-d*s[1])
        N=(N[1],N[0]%N[1])
    assert N[0]==1# gcd(a,n)!=1
    return r[0]%n

def modexp(a,m,n):#computes a**m%n by repeated squaring
    if m==1:
        return a%n
    elif m%2 == 0:
        return modexp((a*a)%n,m//2,n)
    else:
        return (a*modexp((a*a)%n,(m-1)//2,n))%n
    
def lcmB(B):#computes lcm of 1,2,3,...,B
    product = 1
    for p in primes(B):
        product *= p**(int(log(B ,p))) 
    return product

def strongTrial(n,a):#use strong test
    #n-1=2^st
    t=n-1
    s=0
    while t%2==0:
        t//=2
        s+=1
    a=modexp(a,t,n)
    if a==1:
        return True
    for i in range(s):
        if a==n-1:
            return True
        a=(a*a)%n
    return False

def isPrime(n):#use strong test 10 times
    for i in range(10):
        a=randint(1,n-1)
        if gcd(a,n)>1:
            return False
        if not strongTrial(n,a):
            return False
    return True

In [2]:
################################# POLLARD'S P-1 ###################################################

def pollard(N,B=15,m=0):#implements pollards p-1 algorithm
    if not m: #this way we don't compute lcmB(B) many times if we run the algorithm many times
        m = lcmB(B)
    for i in range(100):
        a=randint(2,N-2)
        am_1 = modexp(a,m,N) - 1
        factor = gcd(am_1,N)
        if 1 < factor < N:
            return factor,N//factor
    return -1

In [3]:
#Testing pollard's p-1

print(pollard(134567))
print(pollard(53253235))
print(pollard(2269*3343)) #Testing a semi prime, note that it often fails so returns -1
print(pollard(35327*34537))

(53, 2539)
(1055, 50477)
(2269, 3343)
-1


In [4]:
########################### GRAPH FOR B's IN POLLARD ########################################################
from time import time
from matplotlib import pyplot as plt

def pollardGraphB(N):
    B=list(range(3,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=pollard(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

#pollardGraphB(35327*34537)

In [5]:
############################### LENSTRAS METHOD ###########################################################

def ECadd(a,N,P,Q): # add points P & Q over the elliptic curve y^2 = x^3 + ax + 1
    if P == "inf": #case 1: P is infinity
        return Q
    if Q == "inf": #case 2: Q is infinity
        return P
    if P[0] == Q[0] and (P[1] + Q[1])%N==0: #case 3: points on same x line
        return "inf"
    else: #compute gradient, use this to find new (x,y)
        if P==Q:
            grad = ((3*P[0]**2 + a)*(modinv(2*P[1],N)))%N
        else:
            grad = ((P[1]-Q[1])*(modinv(P[0]-Q[0],N)))%N
        x = (grad**2 - P[0]- Q[0])%N #x co-ord of output (R in wstein ENT alg 6.2.1)
        y = (-grad*x-P[1]+grad*P[0])%N
        return (x,y)
           
def lenstraTrial(a,N,m):#does lenstras algorithm on the EC y^2=x^3+ax+1
    P=(0,1)
    mP=(0,1)# use this to compute mP
    m=bin(m)[2:]#this is m in binary as a string of 0s and 1s
    #now calculate m*P
    for bit in m[1:]:#looks at bits but ignores the first to do repeated doubling
        try:
            mP=ECadd(a,N,mP,mP)
        except:#then we have found a nonzero gcd in denom, which is 2P[1] in ECadd
            return gcd(N,mP[1])#y1 is a non-unit in Z/NZ so this is a non-trivial factor
        if bit=='1':
            try:
                mP=ECadd(a,N,mP,P)
            except: #we found a non 0 gcd in denom, which is P[0]-Q[0]=mP[0]
                return gcd(N,mP[0])
    return N#failed to find a non-trivial factor

def lenstra(N,B=15,numTrials=10,m=0):#trying different elliptic curves until one works
    if N%2==0:
        return 2,N//2
    if not m:#if we have pre-computed an m, don't compute it again!
        m=lcmB(B)
    for i in range(numTrials):
        a=randint(1,N-1)
        g=lenstraTrial(a,N,m)
        if 1<g<N:
            return g,N//g
    return -1#failed to find a factor


In [6]:
#Attempting to factorise the same integers as we tested for pollard, notice that here we can in fact factorise the semi prime!

print(lenstra(134567))
print(lenstra(53253235))
print(lenstra(2269*3343))
print(lenstra(35327*34537)) #This one only factorises a some of the time, you can re run it to see.

(53, 2539)
(5, 10650647)
(3343, 2269)
-1


In [7]:
def lenstraGraphB(N):
    B=list(range(3,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=lenstra(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

#lenstraGraphB(35327*34537)

In [15]:
######################### LOG TIME GRAPH FOR POLLARD & LENSTRA ##########################################
from time import time
from matplotlib import pyplot as plt

def timegraph(reps=10000,B=15,lowerbound=10000,upperbound=100000):
    m=lcmB(B)
    tpollard=[]
    pollardfails=0
    tlenstra=[]
    lenstrafails=0
    for i in range(reps):
        n=randint(lowerbound,upperbound)*2+1
        while isPrime(n):
            n=randint(lowerbound,upperbound)*2+1
        start=time()
        for j in range(1):
            f=pollard(n,m=m)
            if f==-1:
                pollardfails+=1
        tpollard.append(log(time()-start))
        start=time()
        for j in range(1):
            f=lenstra(n,m=m)
            if f==-1:
                lenstrafails+=1
        tlenstra.append(log(time()-start))
    plt.hist(tpollard,color='r',alpha=0.5,bins=30)
    plt.hist(tlenstra,color='b',alpha=0.5,bins=30)
    plt.legend(['Pollard','Lenstra'])
    plt.xlabel('log of time')
    plt.ylabel('frequency')
    plt.show()
    print(f"pollard failed {pollardfails} times")
    print(f"lenstra failed {lenstrafails} times")

In [31]:
#log time histogram showing the frequencies of N ’s against time taken to factorise with Pollard's and Lenstra's methods. It shows that Pollard's is slower than Lenstra's which is to be expected.

#timegraph()

In [9]:
########################## DIXONS'S RANDOM SQUARES#########################################
def Bsmoothness(N,primesuptoB): #this is so we can find suitible k's that are B smooth
    output = []
    for p in primesuptoB:
        count = 0
        while N%p==0:
            N/=p
            count +=1
        output.append(count) # adds the power to the list
    if N==1:
        return output 
    return False #fails if not B smooth


def dixons(N,B=10):
    P=list(primes(B))
    goodA=[] #initialise a list of a's that are squares
    goodK=[]
    J = matrix(ZZ)
    lb=int(sqrt(N)) #bounds for our random a's
    ub=N-lb
    for _ in range(1000):
        a=randint(lb,ub)
        k= (a*a)%N
        if k==0: #need to deal with this case separately otherwise Bsmoothness will go on forever
            return gcd(a,N),N/gcd(a,N)
        kFactors = Bsmoothness(k,P)
        if kFactors:
            J=matrix(ZZ,J.rows()+[kFactors]) #adds the row kFactors to the matrix J
            goodA.append(a)
            goodK.append(k)
            x=matrix(GF(2),J).left_kernel().random_element() #takes a random element in the null space of matrix J mod 2
            A,B=1,1 
            for i in range(len(x)): # A is the product of good A's 
                if x[i]==1:
                    A*=goodA[i]
            y=1/2*matrix(ZZ,x)*J #This will give the prime factorisation of B
            for i in range(len(P)): 
                B*=P[i]^y[0,i] #calculates the integer B so now we have A^2=B^2 mod N
            g=gcd(A-B,N)
            if g != 1 and g != N:
                return g,N/g
    return -1
            

In [10]:
#Example of writing 36 as an array as given in Dixon's method

Bsmoothness(36,list(primes(10)))

[2, 2, 0, 0]

In [11]:
#Testing Dixon's random squares

print(dixons(134567,100))
print(dixons(53253235,100))
print(dixons(2269*3343,100))
print(dixons(35327*34537,100))

#As we can see Dixon's method also can factorise this semi prime so it works better than pollard's!
#It also can find the 4th semi-prime which Lenstra's could not

(2539, 53)
(252385, 211)


(2269, 3343)


(35327, 34537)


In [14]:
######################## GRAPH FOR WHAT VALUES OF B WORK WELL FOR DIXONS ################################

def dixonGraphB(N):
    B=list(range(3,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=dixons(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

#dixonGraphB(35327*34537)

In [16]:
############################ CONTINUED FRACTIONS ###############################
maxima_calculus('algebraic: true;') # So that sage stores the (a+b sqrt(N))/d  in simplified form
def cFracFac(N, B=10):
    Ns = sqrt(N)  # Start with the initial value of Ns
    if Ns.is_integer():  # If Ns is an integer, we have found a square
        return Ns, Ns
    As = floor(Ns)  # The floor of Ns
    p = (1, As)  # p starts as [1, As]
    P = list(primes(B))
    goodP = []  # Initialize a list of p's that are squares
    goodK = []
    J = matrix(ZZ)  # Empty matrix to hold factors
    for i in range(1000): #This is how many convergents we will take 
        k = p[1]**2 % N  # Calculate k
        if k > N / 2:
            k -= N  # Adjust k if it's greater than N/2
        kFactors = Bsmoothness(abs(k), P)
        if kFactors:
            kFactors = [int(k < 0)] + kFactors  # Handle the case where k is negative
            J = matrix(ZZ, J.rows() + [kFactors])  # Add row kFactors to matrix J
            goodP.append(p[-1])  # Add the current p to goodP
            goodK.append(k)  # and k to goodK
            x = matrix(GF(2), J).left_kernel().random_element()  # Random element in the null space mod 2, so this may produce a good A and B
            A, B = 1, 1  # Initialize A and B
            for i in range(len(x)):  # Compute A as the product of good A's
                if x[i] == 1:
                    A *= goodP[i]
            y = 1 / 2 * matrix(ZZ, x) * J  # Compute B from the kernel
            for i in range(len(P)):  # Calculate B using the primes in P
                B *= P[i] ** y[0, i + 1]
            g = gcd(A - B, N)
            if g != 1 and g != N:
                return g, N / g  # Return the factors if a non-trivial gcd is found
        Ns = (1 / (Ns - As)).simplify_rational()  # Update Ns
        As = floor(Ns)  # Update As
        p = (p[1],(p[1] * As + p[0]) % N)  # Update p
    return -1

In [8]:
######################## GRAPH FOR WHAT VALUES OF B WORK WELL FOR CFRAC ################################

def cFracGraphB(N):
    B=list(range(50,200,2))
    goodB=[]
    goodT=[]
    badB=[]
    badT=[]
    for i in B:
        start=time()
        m=cFracFac(N,i)
        end=time()
        if m==-1:
            badB.append(i)
            badT.append(end-start)
        else:
            goodB.append(i)
            goodT.append(end-start)
    plt.scatter(goodB,goodT,color='g')
    plt.scatter(badB,badT,color='r')
    plt.xlabel('Value of B')
    plt.ylabel('Time taken to factorise (or not factorise) N in seconds')
    plt.legend(['Success','Fail'], loc='upper left')
    plt.show()

#cFracGraphB(35327*34537)

In [16]:
#Testing Continued Fracion


print(cFracFac(134567,100))
print(cFracFac(53253235,100))
print(cFracFac(2269*3343,100))
print(cFracFac(35327*34537,100)) #if we choose B=10 this doesn't work, so I chose a bigger value of B

(2539, 53)
(1521521, 35)


(3343, 2269)


(35327, 34537)


In [17]:
###################### LOG TIME GRAPH FOR DIXONS & CONTINUED FRACTIONS ##################################
#This takes too long for my computer...

def timegraph2(reps=300,B=200,lowerbound=1000,upperbound=10000):
    tdixons=[]
    tcFrac=[]
    dixonfails=0
    cfracfails=0
    for i in range(reps):
        print(i)
        n=randint(lowerbound,upperbound)*2+1
        while isPrime(n):
            n=randint(lowerbound,upperbound)*2+1
        start=time()
        for j in range(1):
            f=dixons(n,B)
            if f==-1:
                dixonfails+=1
        tdixons.append(log(time()-start))
        start=time()
        for j in range(1):
            f=cFracFac(n,B)
            if f==-1:
                cfracfails+=1
        tcFrac.append(log(time()-start))
    plt.hist(tdixons,color='r',alpha=0.5,bins=30)
    plt.hist(tcFrac,color='b',alpha=0.5,bins=30)
    plt.legend(['Dixons','Continued Fractions'])
    plt.xlabel('log of time')
    plt.ylabel('frequency')
    print(f"Dixon failed {dixonfails} times")
    print(f"Continued Fraction failed {cfracfails} times")
    plt.show()
    

In [13]:
#Before you run this, change i=100 in the CFRAC method, this is so that the computer can actual produce this graph for this many repetitions

timegraph2()